# Algo de recommandation

In [26]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

In [27]:
def recommand_product(df, product_name, tsne_cols=['tsne1', 'tsne2', 'tsne3'], cluster_col='cluster'):
    """
    Recommande un produit le plus proche dans chaque cluster pour un produit donné.

    Paramètres :
        df (pd.DataFrame) : Le DataFrame contenant les colonnes t-SNE, cluster, et product_name.
        product_name (str) : Le nom du produit choisi.
        tsne_cols (list) : Colonnes représentant les coordonnées dans l'espace vectoriel.
        cluster_col (str) : Colonne représentant les clusters.

    Retourne :
        pd.DataFrame : Les produits recommandés (le plus proche dans chaque cluster).
    """
    # Vérifier si le produit existe dans le DataFrame
    if product_name not in df['product_name'].values:
        raise ValueError(f"Le produit {product_name} n'existe pas dans le DataFrame.")
    
    # Extraire les informations du produit choisi
    chosen_product = df[df['product_name'] == product_name]
    chosen_cluster = chosen_product[cluster_col].values[0]
    chosen_coords = chosen_product[tsne_cols].values[0]

    # Initialiser une liste pour les recommandations
    recommendations = []

    # Parcourir les clusters (sauf celui du produit choisi)
    for cluster in df[cluster_col].unique():
        
        # Produits appartenant au cluster actuel
        cluster_products = df[df[cluster_col] == cluster]
                
        if cluster == chosen_cluster:
            # Dans le même cluster, exclure le produit choisi
            cluster_products = cluster_products[cluster_products['product_name'] != product_name]
        
        # Calculer la distance entre le produit choisi et les produits de ce cluster
        distances = cdist([chosen_coords], cluster_products[tsne_cols].values, metric='euclidean')
        
        # Trouver l'index du produit le plus proche
        closest_index = np.argmin(distances)
        
        # Ajouter le produit le plus proche à la liste des recommandations
        recommendations.append(cluster_products.iloc[closest_index])
    
    # Convertir les recommandations en DataFrame
    recommendations_df = pd.DataFrame(recommendations)
    return recommendations_df

# Test sur le dataset

In [28]:
df = pd.read_csv('./amazon_clusters.csv')
produit = df.sample(1).values[0]
nom_produit, cluster_produit = produit[2], produit[0]
print(f"Produit choisi : {nom_produit} (cluster {cluster_produit})")

Produit choisi : Havells Ambrose 1200mm Ceiling Fan (Gold Mist Wood) (cluster 4)


In [29]:
df_recommdations = recommand_product(df, nom_produit)
df_recommdations

,cluster,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,...,review_title,review_content,img_link,product_link,stock,embedding,text,tsne1,tsne2,tsne3
52,2,B082T6V3DT,AmazonBasics New Release Nylon USB-A to Lightn...,Computers&Accessories|Accessories&Peripherals|...,₹799,"₹2,100",62%,4.3,"8,188",Added Protection: An additional layer of prote...,...,"Good product but costly,It’s really long n stu...","It cost should be under Rs. 500,Buy it,Color i...",https://m.media-amazon.com/images/I/31kw1RgU5y...,https://www.amazon.in/AmazonBasics-Nylon-Braid...,90,"[-0.041015625, 0.0251312255859375, 0.031204223...",product_name amazonbasics new release nylon us...,7.272638,6.195969,-7.185114
132,5,B08L4SBJRY,Saifsmart Outlet Wall Mount Hanger Holder for ...,Electronics|HomeAudio|Accessories|SpeakerAcces...,₹349,"₹1,299",73%,4.0,"3,295",Hand Free,...,"Good Quality but cheap color,Good product ,at ...",The color is not same as what is shown in imag...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/SAIFSMART-Compact-Bracke...,106,"[-0.024169921875, 0.0088043212890625, 0.011062...",product_name saifsmart outlet wall mount hange...,-2.166292,-1.684141,-6.986373
298,0,B09HN7LD5L,PROLEGEND® PL-T002 Universal TV Stand Table To...,"Electronics|HomeTheater,TV&Video|Accessories|T...","₹1,850","₹4,500",59%,4.0,184,TV LEG】--The universal tv stand base fits most...,...,"A Good Product.,Does the job,Overpriced Item,A...","A good product.,Got this for old 40 inch LG tv...",https://m.media-amazon.com/images/I/21rEkD8xxp...,https://www.amazon.in/PROLEGEND%C2%AE-PL-T002-...,112,"[-0.052978515625, 0.0217437744140625, 0.008850...",product_name prolegend plt002 universal tv sta...,-5.550643,-6.047413,-9.969180
571,1,B0B9BXKBC7,"WeCool S5 Long Selfie Stick, with Large Reinfo...",Electronics|Mobiles&Accessories|MobileAccessor...,"₹1,799","₹3,999",55%,4.6,245,"64"" Tall Cell Phone Tripods with 7 section adj...",...,"Good,Good Product but Little expensive.,Happy ...","Good,Thoda sa kam stable hai, phone lagane ke ...",https://m.media-amazon.com/images/I/41fDM4QUfv...,https://www.amazon.in/WeCool-Reinforced-Functi...,102,"[-0.03302001953125, -0.001758575439453125, 0.0...",product_name wecool s5 long selfie stick large...,-4.258567,-5.077892,1.047095
1157,3,B0912WJ87V,Reffair AX30 [MAX] Portable Air Purifier for C...,Car&Motorbike|CarAccessories|InteriorAccessori...,"₹2,339","₹4,000",42%,3.8,"1,118",[NEW LAUNCH - INNOVATIVE DESIGN] EFFICIENT PUR...,...,"Liked it,User friendly and effective,Not as pe...","As of now its working fine, but don't expect m...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Reffair-AX30-MAX-Interna...,81,"[-0.04107666015625, 0.032745361328125, 0.04141...",product_name reffair ax30 max portable air pur...,-11.947196,7.060156,-10.492670
1442,4,B01L7C4IU2,Havells Ambrose 1200mm Ceiling Fan (Pearl Whit...,"Home&Kitchen|Heating,Cooling&AirQuality|Fans|C...","₹2,199","₹3,045",28%,4.2,"2,686",Installation is not provided by brand; For pro...,...,"Very good product..quality is good,A good prod...","Nice color..,Overall I am satisfied with the p...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Havells-Ambrose-1200mm-C...,94,"[-0.0672607421875, 0.0017576217651367188, 0.03...",product_name havells ambrose 1200mm ceiling fa...,-16.333261,3.367528,-14.357140
